# Biber Resultate Pre-Processing

- Download Raw Data from https://wettbewerb.informatik-biber.ch/admin/
- Konvertiere Excel-Tabs zu CSV-Files mittels "Save As" in Excel, UTF8-kodiert
- Verwende `biber-preprocess.raku pfad/zu/csv/dateien/` zum Anonymisieren der gesamten Rohdaten
- Kopiere anonymisierte Rohdaten nach data/
- Verwende dieses Notebook, um ein Biber-Long-Format.csv zu erzeugen, in der alle Rohdaten enthalten sind.
- Schülerinnen und Schüler arbeiten mit Biber-Long-Format.csv

In [ ]:
import pandas as pd

## Multi CSV Files zu Long-Format

In [ ]:
data_2024a = pd.read_csv("data/2024a.csv")
data_2025a = pd.read_csv("data/2025a.csv")
data_2025b = pd.read_csv("data/2025b.csv")

## Daten zusammenführen

In den verschiedenen Jahren gibt es unterschiedliche Aufgaben. Wir überführen die Daten deshalb in ein anderes Format - das sogenannte Long-Format - so dass alle Jahre in den selben Dataframe gespeichert und dann gemeinsam analysiert werden können.

|Jahr|Land|Schule|Klassenstufe|Klasse|Name|Team|Aufgabe|Punkte|
|--|--|--|--|--|--|--|--|--|
|2025|NaN|NaN|Schuljahre 9 und 10|MAK|kygrj|NaN|Bebrasien|-2|
|...|...|...|...|...|...|...|...|...|

In [ ]:
data_2024a_long = pd.melt(
    data_2024a,
    id_vars=['Jahr', 'Land', 'Schule', 'Klassenstufe', 'Klasse', 'Name', 'Team'],     # columns that stay as identifiers
    var_name='Aufgabe',   # name for the new "variable" column
    value_name='Punkte'     # name for the new "value" column
)

In [ ]:
data_2025a_long = pd.melt(
    data_2025a,
    id_vars=['Jahr', 'Land', 'Schule', 'Klassenstufe', 'Klasse', 'Name', 'Team'],
    var_name='Aufgabe',
    value_name='Punkte'
)

In [ ]:
data_2025b_long = pd.melt(
    data_2025b,
    id_vars=['Jahr', 'Land', 'Schule', 'Klassenstufe', 'Klasse', 'Name', 'Team'],
    var_name='Aufgabe',
    value_name='Punkte'
)

Jetzt müssen wir die drei Dataframes für die Analyse zusammenführen. In diesem Fall also einfach untereinander schreiben, da sie die gleichen Spaltennamen haben.

In [ ]:
df = pd.concat([data_2024a_long, data_2025a_long, data_2025b_long])

Wir entfernen die Bonuspunkte. Diese Punkte bekommt jede(r) Teilnehmer(in) und sie dienen nur dazu, dass kein Teilnehmer weniger als 0 Punkte erreichen kann. Wir benötigen diese nicht für die Analyse - im Gegenteil, die Bonuspunkte stören sogar.

In [ ]:
df = df[df["Aufgabe"] != "Bonus"]

In [ ]:
df.describe()

In [ ]:
df.to_csv("data/Biber-Daten-Long.csv", index=False)
df.to_excel("data/Biber-Daten-Long.xlsx", index=False)

und nochmal im Wide-Format.

In [ ]:
df_wide = df.pivot(
    index=['Jahr', 'Land', 'Schule', 'Klassenstufe', 'Klasse', 'Name', 'Team'], # neue Zeilen
    columns="Aufgabe",
    values="Punkte",
).reset_index()

In [ ]:
df_wide.describe()

In [ ]:
df_wide

In [ ]:
df_wide.to_csv("data/Biber-Daten-Wide.csv", index=False)
df_wide.to_excel("data/Biber-Daten-Wide.xlsx", index=False)

### Bilder als Dateien abspeichern

In [ ]:
img = df.boxplot(column='Punkte', by='Jahr', showmeans=True, grid=False)
img.get_figure().savefig('img/a_boxplot.svg')
img.get_figure().savefig('img/a_boxplot.png')